- Luis Henrique Caldas Altero – RM88670
- Pedro Guilherme Polloni Barreto - RM 88964



In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from statsmodels.api import OLS
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
# Fizemos o carregamento do dataset
df = pd.read_csv('venda.csv')

In [3]:
# Printamos as primeiras linhas para estudar o dataset
df.head()

,Unnamed: 0,X,date,mes,weekday,margem,venda,desconto,outdesc,outmg
0,1,1,20140801,agosto,sexta-feira,0.406111,110042.460,9190.907534,0,0
1,2,2,20140802,agosto,sabado,0.416022,58377.320,5713.043012,0,0
2,3,3,20140803,agosto,domingo,0.431993,64635.390,8621.708915,0,0
3,4,4,20140804,agosto,segunda-feira,0.409216,140417.321,18312.965640,0,0
4,5,5,20140805,agosto,terca-feira,0.449648,149700.286,19942.741300,0,0


In [4]:
# Criamos uma função para verificarmos os valores NaN no dataset
def valores_nan(df):
  valores_nan = []
  for coluna in df.columns:
    contagem_nan = df.loc[pd.isna(df[coluna]), coluna].shape[0]
    valores_nan.append(contagem_nan)
  return valores_nan

In [5]:
# Verificamos que a coluna Target possui 31 valores nulos
valores_nan(df)

[0, 0, 0, 0, 0, 0, 31, 0, 0, 0]

In [6]:
# Optamos por deletar os valores nulos por se tratar da coluna Target 
df = df[ ~pd.isna(df.venda)]

In [7]:
# Executamos a função novamente para verificar se os valores realmente foram dropados
valores_nan(df)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [8]:
# Deletamos as colunas 0 e X
del df['Unnamed: 0']
del df['X']

In [9]:
# Função que transforma dados categóricos em dummies de acordo com o tipo de dado
def create_dummies(dataframe):
  column_type = dataframe.dtypes
  num = 0
  for column in dataframe.columns:
    if str(column_type[num]) == 'object':
      df_dummies = pd.get_dummies(dataframe[[column]])
      for values in df_dummies.columns:
        dataframe[values] = df_dummies[values]
      del dataframe[column]
      num += 1
    else:
      num += 1
  return dataframe

In [10]:
# Executamos a função de criação de dummies
create_dummies(df)

,date,margem,venda,desconto,outdesc,outmg,mes_abril,mes_agosto,mes_dezembro,mes_fevereiro,...,mes_novembro,mes_outubro,mes_setembro,weekday_domingo,weekday_quarta-feira,weekday_quinta-feira,weekday_sabado,weekday_segunda-feira,weekday_sexta-feira,weekday_terca-feira
0,20140801,0.406111,110042.460,9190.907534,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,20140802,0.416022,58377.320,5713.043012,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,20140803,0.431993,64635.390,8621.708915,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
3,20140804,0.409216,140417.321,18312.965640,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,20140805,0.449648,149700.286,19942.741300,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,20150727,0.394510,43796.980,3487.843143,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
361,20150728,0.480154,71934.730,4965.290000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
362,20150729,0.446441,124043.650,3477.950004,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
363,20150730,0.429696,47482.530,4328.840013,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [11]:
# Verificamos o tamanho do dataset para separarmos a base de treino e teste
df.shape

(365, 25)

In [12]:
# Separamos o teste em 30% do total
treino = df.iloc[0:255]
teste  = df.iloc[256:365]

In [13]:
# Definimos o X e Y treino
Y_treino = treino['venda']
del treino['venda']
X_treino = treino

In [14]:
# Definimos o X e Y teste
Y_teste = teste['venda']
del teste['venda']
X_teste = teste

- Modelo 1: Utilizamos todas as colunas no modelo

In [15]:
# Executamos o modelo de Regreção linear devido ao target ser numérico
mod = LinearRegression()
mod = mod.fit(X_treino, Y_treino)
Y_prev = mod.predict(X_teste)

In [16]:
# Calculamos o erro de previsão
erro = Y_teste - Y_prev

# Calculo do erro absoluto médio
erro_abs_medio = np.mean(abs(erro))
print(f'Erro absoluto médio: {erro_abs_medio}')

# Calculo do erro relativo (percentual)
erro_medio_perc = erro_abs_medio/np.mean(Y_teste)
print(f'Erro relativo (percentual): {erro_medio_perc}')

Erro absoluto médio: 1527674.172504711
Erro relativo (percentual): 28.72757372845627


In [17]:
OLS(Y_treino, X_treino).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  venda   R-squared:                       0.853
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     71.75
Date:                Fri, 10 Jun 2022   Prob (F-statistic):           1.79e-86
Time:                        00:30:08   Log-Likelihood:                -2842.5
No. Observations:                 255   AIC:                             5725.
Df Residuals:                     235   BIC:                             5796.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
date                   -327.3262    129.850     -2.521      0.012    -583.144     -71.508
margem                -5.479e+05   5.14e+04    -10.662      0.000   -6.49e+05   -4.47e+05
desconto                  7.6255      0.355     21.456      0.000       6.925       8.326
outdesc                4.423e+04   1.61e+04      2.754      0.006    1.26e+04    7.59e+04
outmg                 -3.581e+04   1.35e+04     -2.661      0.008   -6.23e+04   -9296.180
mes_abril              2.887e+09   1.15e+09      2.521      0.012    6.31e+08    5.14e+09
mes_agosto             2.884e+09   1.14e+09      2.521      0.012     6.3e+08    5.14e+09
mes_dezembro           2.884e+09   1.14e+09      2.521      0.012     6.3e+08    5.14e+09
mes_fevereiro          2.887e+09   1.15e+09      2.521      0.012    6.31e+08    5.14e+09
mes_janeiro            2.887e+09   1.15e+09      2.521      0.012    6.31e+08    5.14e+09
mes_julho                -0.2114      0.084     -2.521      0.012      -0.377      -0.046
mes_junho                -0.7136      0.283     -2.521      0.012      -1.271      -0.156
mes_maio                  0.5647      0.224      2.521      0.012       0.123       1.006
mes_mar?o              2.887e+09   1.15e+09      2.521      0.012    6.31e+08    5.14e+09
mes_novembro           2.884e+09   1.14e+09      2.521      0.012     6.3e+08    5.14e+09
mes_outubro            2.884e+09   1.14e+09      2.521      0.012     6.3e+08    5.14e+09
mes_setembro           2.884e+09   1.14e+09      2.521      0.012     6.3e+08    5.14e+09
weekday_domingo        3.709e+09   1.47e+09      2.521      0.012     8.1e+08    6.61e+09
weekday_quarta-feira   3.709e+09   1.47e+09      2.521      0.012     8.1e+08    6.61e+09
weekday_quinta-feira   3.709e+09   1.47e+09      2.521      0.012     8.1e+08    6.61e+09
weekday_sabado         3.709e+09   1.47e+09      2.521      0.012     8.1e+08    6.61e+09
weekday_segunda-feira  3.709e+09   1.47e+09      2.521      0.012     8.1e+08    6.61e+09
weekday_sexta-feira    3.709e+09   1.47e+09      2.521      0.012     8.1e+08    6.61e+09
weekday_terca-feira    3.709e+09   1.47e+09      2.521      0.012     8.1e+08    6.61e+09
==============================================================================
Omnibus:                       49.756   Durbin-Watson:                   1.463
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              646.169
Skew:                           0.129   Prob(JB):                    4.85e-141
Kurtosis:                      10.794   Cond. No.                     2.13e+24
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.27e-32. This might indicate that there are
str

- Modelo 2: Utilizamos apenas as colunas  ['date', 'margem', 'desconto', 'outdesc',	'outmg'] que são os dados numéricos do dataset

In [18]:
# Executamos o modelo de Regreção linear devido ao target ser numérico
mod = LinearRegression()
mod = mod.fit(X_treino.loc[:, ['date', 'margem', 'desconto', 'outdesc',	'outmg']], Y_treino)
Y_prev = mod.predict(X_teste.loc[:, ['date', 'margem', 'desconto', 'outdesc',	'outmg']])

In [19]:
# Calculamos o erro de previsão
erro = Y_teste - Y_prev

# Calculo do erro absoluto médio
erro_abs_medio = np.mean(abs(erro))
print(f'Erro absoluto médio: {erro_abs_medio}')
# Calculo do erro relativo (percentual)
erro_medio_perc = erro_abs_medio/np.mean(Y_teste)
print(f'Erro relativo (percentual): {erro_medio_perc}')

Erro absoluto médio: 15081.860207245138
Erro relativo (percentual): 0.2836110335985728


In [20]:
OLS(Y_treino, X_treino.loc[:, ['date', 'margem', 'desconto', 'outdesc',	'outmg']]).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  venda   R-squared (uncentered):                   0.956
Model:                            OLS   Adj. R-squared (uncentered):              0.956
Method:                 Least Squares   F-statistic:                              1098.
Date:                Fri, 10 Jun 2022   Prob (F-statistic):                   7.22e-168
Time:                        00:30:10   Log-Likelihood:                         -2902.7
No. Observations:                 255   AIC:                                      5815.
Df Residuals:                     250   BIC:                                      5833.
Df Model:                           5                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
date           0.0116      0.001     14.089      0.000       0.010       0.013
margem     -5.011e+05   4.29e+04    -11.686      0.000   -5.85e+05   -4.17e+05
desconto       7.4043      0.338     21.936      0.000       6.740       8.069
outdesc     5.217e+04   1.81e+04      2.875      0.004    1.64e+04    8.79e+04
outmg       -5.96e+04   1.58e+04     -3.768      0.000   -9.07e+04   -2.85e+04
==============================================================================
Omnibus:                       27.764   Durbin-Watson:                   1.163
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              143.034
Skew:                           0.004   Prob(JB):                     8.72e-32
Kurtosis:                       6.669   Cond. No.                     6.50e+08
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The condition number is large, 6.5e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

- Modelo 3: Utilizamos apenas as colunas ['weekday_sabado', 'weekday_sexta-feira', 'weekday_domingo', 'mes_janeiro', 'mes_dezembro', 'mes_fevereiro', 'mes_julho', 'date', 'margem', 'desconto', 'outdesc',	'outmg']. Separando as colunas em períodos que as pessoas provavelmente pedem bebidas.

In [21]:
# Executamos o modelo de Regreção linear devido ao target ser numérico
mod = LinearRegression()
mod = mod.fit(X_treino.loc[:, ['weekday_sabado', 'weekday_sexta-feira', 'weekday_domingo', 'mes_janeiro', 'mes_dezembro', 'mes_fevereiro', 'mes_julho', 'date', 'margem', 'desconto', 'outdesc', 'outmg']], Y_treino)
Y_prev = mod.predict(X_teste.loc[:, ['weekday_sabado', 'weekday_sexta-feira', 'weekday_domingo', 'mes_janeiro', 'mes_dezembro', 'mes_fevereiro', 'mes_julho', 'date', 'margem', 'desconto', 'outdesc', 'outmg']])

In [22]:
# Calculamos o erro de previsão
erro = Y_teste - Y_prev

# Calculo do erro absoluto médio
erro_abs_medio = np.mean(abs(erro))
print(f'Erro absoluto médio: {erro_abs_medio}')
# Calculo do erro relativo (percentual)
erro_medio_perc = erro_abs_medio/np.mean(Y_teste)
print(f'Erro relativo (percentual): {erro_medio_perc}')

Erro absoluto médio: 14965.868811592072
Erro relativo (percentual): 0.28142984114898995


In [23]:
OLS(Y_treino, X_treino.loc[:, ['weekday_sabado', 'weekday_sexta-feira', 'weekday_domingo', 'mes_janeiro', 'mes_dezembro', 'mes_fevereiro', 'mes_julho', 'date', 'margem', 'desconto', 'outdesc', 'outmg']]).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  venda   R-squared (uncentered):                   0.966
Model:                            OLS   Adj. R-squared (uncentered):              0.964
Method:                 Least Squares   F-statistic:                              629.3
Date:                Fri, 10 Jun 2022   Prob (F-statistic):                   3.63e-172
Time:                        00:30:11   Log-Likelihood:                         -2871.4
No. Observations:                 255   AIC:                                      5765.
Df Residuals:                     244   BIC:                                      5804.
Df Model:                          11                                                  
Covariance Type:            nonrobust                                                  
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
weekday_sabado      -1.812e+04   3718.065     -4.872      0.000   -2.54e+04   -1.08e+04
weekday_sexta-feira -5440.0877   3579.634     -1.520      0.130   -1.25e+04    1610.840
weekday_domingo     -1.573e+04   3649.528     -4.310      0.000   -2.29e+04   -8541.376
mes_janeiro         -2642.0376   4176.733     -0.633      0.528   -1.09e+04    5585.015
mes_dezembro        -2.512e+04   4516.060     -5.562      0.000    -3.4e+04   -1.62e+04
mes_fevereiro        -879.9262   4141.488     -0.212      0.832   -9037.557    7277.704
mes_julho            2.581e-10   2.01e-11     12.863      0.000    2.19e-10    2.98e-10
date                    0.0134      0.001     14.497      0.000       0.012       0.015
margem              -5.564e+05   4.63e+04    -12.021      0.000   -6.48e+05   -4.65e+05
desconto                6.8613      0.324     21.144      0.000       6.222       7.500
outdesc              6.097e+04   1.66e+04      3.677      0.000    2.83e+04    9.36e+04
outmg               -4.541e+04   1.44e+04     -3.149      0.002   -7.38e+04    -1.7e+04
==============================================================================
Omnibus:                       51.015   Durbin-Watson:                   1.283
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              728.442
Skew:                           0.047   Prob(JB):                    6.62e-159
Kurtosis:                      11.280   Cond. No.                     1.70e+24
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The smallest eigenvalue is 3.57e-32. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

- Modelo 4: Utilizamos apenas as colunas ['weekday_sabado', 'weekday_sexta-feira', 'weekday_domingo', 'mes_dezembro', 'mes_julho', 'date', 'margem', 'desconto', 'outdesc', 'outmg'], retiramos o mês de janeiro  e fevereiro devido ao P valor mostrado anteriormente.

In [24]:
# Executamos o modelo de Regreção linear devido ao target ser numérico
mod = LinearRegression()
mod = mod.fit(X_treino.loc[:, ['weekday_sabado', 'weekday_sexta-feira', 'weekday_domingo', 'mes_dezembro', 'mes_julho', 'date', 'margem', 'desconto', 'outdesc', 'outmg']], Y_treino)
Y_prev = mod.predict(X_teste.loc[:, ['weekday_sabado', 'weekday_sexta-feira', 'weekday_domingo', 'mes_dezembro', 'mes_julho', 'date', 'margem', 'desconto', 'outdesc', 'outmg']])

In [25]:
# Calculamos o erro de previsão
erro = Y_teste - Y_prev

# Calculo do erro absoluto médio
erro_abs_medio = np.mean(abs(erro))
print(f'Erro absoluto médio: {erro_abs_medio}')
# Calculo do erro relativo (percentual)
erro_medio_perc = erro_abs_medio/np.mean(Y_teste)
print(f'Erro relativo (percentual): {erro_medio_perc}')

Erro absoluto médio: 14311.507037289555
Erro relativo (percentual): 0.26912471322662795


In [26]:
OLS(Y_treino, X_treino.loc[:, ['weekday_sabado', 'weekday_sexta-feira', 'weekday_domingo', 'mes_dezembro', 'mes_julho', 'date', 'margem', 'desconto', 'outdesc', 'outmg']]).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  venda   R-squared (uncentered):                   0.966
Model:                            OLS   Adj. R-squared (uncentered):              0.965
Method:                 Least Squares   F-statistic:                              774.2
Date:                Fri, 10 Jun 2022   Prob (F-statistic):                   5.73e-175
Time:                        00:30:11   Log-Likelihood:                         -2871.6
No. Observations:                 255   AIC:                                      5761.
Df Residuals:                     246   BIC:                                      5793.
Df Model:                           9                                                  
Covariance Type:            nonrobust                                                  
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
weekday_sabado       -1.82e+04   3699.952     -4.918      0.000   -2.55e+04   -1.09e+04
weekday_sexta-feira -5493.1089   3566.224     -1.540      0.125   -1.25e+04    1531.118
weekday_domingo     -1.567e+04   3633.579     -4.313      0.000   -2.28e+04   -8516.418
mes_dezembro         -2.41e+04   4152.956     -5.803      0.000   -3.23e+04   -1.59e+04
mes_julho            1.852e-10   2.03e-11      9.121      0.000    1.45e-10    2.25e-10
date                    0.0131      0.001     16.073      0.000       0.012       0.015
margem               -5.45e+05   4.24e+04    -12.848      0.000   -6.29e+05   -4.61e+05
desconto                6.8705      0.319     21.549      0.000       6.242       7.498
outdesc              6.136e+04   1.65e+04      3.722      0.000    2.89e+04    9.38e+04
outmg                -4.46e+04   1.43e+04     -3.116      0.002   -7.28e+04   -1.64e+04
==============================================================================
Omnibus:                       51.670   Durbin-Watson:                   1.279
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              758.473
Skew:                           0.051   Prob(JB):                    1.99e-165
Kurtosis:                      11.448   Cond. No.                     9.60e+23
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[3] The smallest eigenvalue is 1.12e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

# Questão 3

1- Qual das duas matrizes de confusão apresentação a maior acurácia? (responda
incluindo o cálculo)

- Resp: As duas apresentam a mesma acurácia.

In [27]:
def acuracia(TP, TN, FN, FP):
  return (TP+TN)/(TP + TN + FN + FP)

In [28]:
print('Acurácia A: ' + str(acuracia(TP=30, TN=140, FN=20, FP=20)))

Acurácia A: 0.8095238095238095


In [29]:
print('Acurácia B: ' + str(acuracia(TP=20, TN=150, FN=30, FP=10)))

Acurácia B: 0.8095238095238095


2- Como vocês podem interpretar o problema de negócio e descrever o que significam
os números FN e FP, em termos de impacto no churn, tanto na Matriz A quanto na B.

- Resp: O FN corresponde aos erros em que a máquina previu que o cliente não abandonou o serviço porém ele abandonou e o FP corresponde aos erros em que a máquina previu que o cliente abandonou o serviço porém ele não abandonou. Vimos que na Matriz A houve um empate de FP e FN, ou seja, a máquina errou na mesma proporção no FP e FN, já na matriz B a máquina obteve mais erros em prever o FN.

3- Se quisermos gerar uma campanha que tenha maior impacto nos clientes que vão
abandonar nosso e-commerce, qual dos 2 algoritmos (qual das 2 matrizes) deveríamos
utilizar? Explique com detalhes.

- Resp: Utilizariamos a matriz A devido a seu recall ter um valor maior (60%), isso mostra que 60% das previsões de saída do cliente estavam certas, enquanto na outra matriz 40% delas estavam certas.

4- Se gerarmos uma 3a matriz de confusão, qual dos erros (FN ou FP) vocês gostariam
que fossem menores e porque? Expliquem com argumentações do problema de churn.

- Resp: Gostaríamos que o FP fosse menor, pois ele mostra quantos clientes o modelo previu que que saíram mas na verdade eles continuam na plataforma. Isso pode causar descontentamento do cliente e perda de fidelidade.

5- Calcule a métrica de Precision (P = TP/(TP+FP)) e Recall (R= TP/(TP+FN)) e interprete
estes % em termos do problema de negócio.

- É possível perceber que a recall da matriz A obteve uma porcentagem maior, isso indica que em problemas em que é preciso ter uma maior acertividade nos clientes que sairam, utilizariamos esse modelo. Agora se for preciso um maior precision para evitar a perda de fidelidade, utilizariamos a matriz B.

In [30]:
def precision(TP, FP):
  return TP/(TP+FP)

def recall(TP, FN):
  return TP/(TP+FN)

In [31]:
print('Precision A: ' + str(precision(TP=30, FP=20)))
print('Recall A: ' + str(recall(TP=30, FN=20)))

Precision A: 0.6
Recall A: 0.6


In [32]:
print('Precision B: ' + str(precision(TP=20, FP=10)))
print('Recall B: ' + str(recall(TP=20, FN=30)))

Precision B: 0.6666666666666666
Recall B: 0.4
